In [205]:
### ModulIDs ###
""" '1223' - Stromerzeugung: Braunkohle
'1224' - Stromerzeugung: Kernenergie
'1225' - Stromerzeugung: Wind Offshore
'1226' - Stromerzeugung: Wasserkraft
'1227' - Stromerzeugung: Sonstige Konventionelle
'1228' - Stromerzeugung: Sonstige Erneuerbare
'4066' - Stromerzeugung: Biomasse
'4067' - Stromerzeugung: Wind Onshore
'4068' - Stromerzeugung: Photovoltaik
'4069' - Stromerzeugung: Steinkohle
'4070' - Stromerzeugung: Pumpspeicher
'4071' - Stromerzeugung: Erdgas
'410' - Stromverbrauch: Gesamt (Netzlast)
'4359' - Stromverbrauch: Residuallast
'4387' - Stromverbrauch: Pumpspeicher """

# power generation 
REALIZED_POWER_GENERATION    = [1001224,1004066,1004067,1004068,1001223,1004069,1004071,1004070,1001226,1001228,1001227,1001225]
INSTALLED_POWER_GENERATION   = [3004072,3004073,3004074,3004075,3004076,3000186,3000188,3000189,3000194,3000198,3000207,3003792]
FORECASTED_POWER_GENERATION  = [2000122, 2000715, 2000125, 2003791, 2000123]

# power consumption
FORECASTED_POWER_CONSUMPTION = [6000411, 6004362]
REALIZED_POWER_CONSUMPTION   = [5000410, 5004359]

# market   [€/MWh]              DE/LUX   DNK1    DNK 2    FR      NL      AT 	  PL     SW4 	  SCH     CZ   DE/AT/LU IT(Nord) Slowenien Ungarn    
WHOLESALE_PRICES             = [8004169,8004170,8000252,8000253,8000251,8000254,8000255,8000256,8000257,8000258,8000259,8000260,8000261,8000262]
COMMERCIAL_FOREIGN_TRADE     = [8004169,8004170,8000252,8000253,8000251,8000254,8000255,8000256,8000257,8000258,8000259,8000260,8000261,8000262]
PHYSICAL_POWER_FLOW          = [31000714,31000140,31000569,31000145,31000574,31000570,31000139,31000568,31000138,31000567,31000146,31000575,31000144,31000573,31000142,31000571,31000143,31000572,31000141]

In [206]:
from deutschland import smard
import time
import requests
from deutschland import smard
from deutschland.smard.api import default_api
from deutschland.smard.model.indices import Indices
from deutschland.smard.model.time_series import TimeSeries
from datetime import datetime
import pandas as pd
from io import BytesIO
url = 'https://www.smard.de/nip-download-manager/nip/download/market-data'

In [207]:
start = datetime(2025, 5, 29, 00, 00, 00)
end =  datetime(2025, 5, 30, 0, 0, 0)
#print(int(start.timestamp())* 1000)
#print(datetime.fromtimestamp(start.timestamp()))

In [208]:
payload = {"request_form":[{"format":"CSV","moduleIds":[1004067,1004068,1001225],"region":"DE","timestamp_from":int(start.timestamp())*1000,"timestamp_to":int(end.timestamp())*1000,"type":"discrete","language":"de"}]}
energy = requests.post(url, json = payload)
df_energy = pd.read_csv(BytesIO(energy.content), sep=';')

df_energy.columns=['Time_from','Time_to','Windoffshore','Windonshore','Solar']

df_energy['Windoffshore'] = df_energy['Windoffshore'].str.replace('.', '')
df_energy['Windoffshore'] = df_energy['Windoffshore'].str.replace(',', '.')
df_energy['Windoffshore'] = df_energy['Windoffshore'].str.replace('-', '0')
df_energy['Windoffshore'] = df_energy['Windoffshore'].astype(float)

df_energy['Windonshore'] = df_energy['Windonshore'].str.replace('.', '')
df_energy['Windonshore'] = df_energy['Windonshore'].str.replace(',', '.')
df_energy['Windonshore'] = df_energy['Windonshore'].str.replace('-', '0')
df_energy['Windonshore'] = df_energy['Windonshore'].astype(float)

df_energy['Solar'] = df_energy['Solar'].str.replace('.', '')
df_energy['Solar'] = df_energy['Solar'].str.replace(',', '.')
df_energy['Solar'] = df_energy['Solar'].str.replace('-', '0')
df_energy['Solar'] = df_energy['Solar'].astype(float)
#df_energy




In [209]:
payload = {"request_form":[{"format":"CSV","moduleIds":[8004169],"region":"DE","timestamp_from":int(start.timestamp())*1000,"timestamp_to":int(end.timestamp())*1000,"type":"discrete","language":"de"}]}
price = requests.post(url, json = payload)
df_price = pd.read_csv(BytesIO(price.content), sep=';')
df_price.columns=['Time_from','Time_to','Market_price']

df_price['Market_price'] = df_price['Market_price'].str.replace('.', '')
df_price['Market_price'] = df_price['Market_price'].str.replace(',', '.')
df_price['Market_price'] = df_price['Market_price'].str.replace('-', '0')
df_price['Market_price'] = df_price['Market_price'].astype(float)

#df_price

In [210]:
payload = {"request_form":[{"format":"CSV","moduleIds":[5000410],"region":"DE","timestamp_from":int(start.timestamp())*1000,"timestamp_to":int(end.timestamp())*1000,"type":"discrete","language":"de"}]}
load = requests.post(url, json = payload)
df_load = pd.read_csv(BytesIO(load.content), sep=';')
df_load.columns=['Time_from','Time_to','Load']
#df_load 
df_load['Load'] = df_load['Load'].str.replace('.', '')
df_load['Load'] = df_load['Load'].str.replace(',', '.')
df_load['Load'] = df_load['Load'].str.replace('-', '0')
df_load['Load'] = df_load['Load'].astype(float)

#df_load

In [211]:
df_inter = pd.merge(df_energy, df_load, on=['Time_from','Time_to'])
df = pd.merge(df_inter, df_price, on=['Time_from'], how = 'outer')
df = df.drop(columns=['Time_to_x', 'Time_to_y'])
#df.columns=['Time','Windoffshore','Windonshore','Solar','Load','Market_price']

df['Wind'] = df['Windoffshore'] + df['Windonshore']

df = df.drop(columns=['Windoffshore', 'Windonshore'])
df['Market_price']= df['Market_price'].ffill()
df.set_index('Time_from')

df.to_csv('SMARD_data.csv', sep='\t',index=False)